# Naive Bayes Algorithm

A machine learning tutorial from https://machinelearningmastery.com/naive-bayes-classifier-scratch-python/

Reimplemented in Scala

## About Naive Bayes
The Naive Bayes algorithm is an intuitive method that uses the probabilities of each attribute belonging to each class to make a prediction. It is the supervised learning approach you would come up with if you wanted to model a predictive modeling problem probabilistically.

Naive bayes simplifies the calculation of probabilities by assuming that the probability of each attribute belonging to a given class value is independent of all other attributes. This is a strong assumption but results in a fast and effective method.

The probability of a class value given a value of an attribute is called the conditional probability. By multiplying the conditional probabilities together for each attribute for a given class value, we have a probability of a data instance belonging to that class.

To make a prediction we can calculate probabilities of the instance belonging to each class and select the class value with the highest probability.

Naive bases is often described using categorical data because it is easy to describe and calculate using ratios. A more useful version of the algorithm for our purposes supports numeric attributes and assumes the values of each numerical attribute are normally distributed (fall somewhere on a bell curve). Again, this is a strong assumption, but still gives robust results.

## Predict the Onset of Diabetes
The test problem we will use in this tutorial is the Pima Indians Diabetes problem.

This problem is comprised of 768 observations of medical details for Pima indians patents. The records describe instantaneous measurements taken from the patient such as their age, the number of times pregnant and blood workup. All patients are women aged 21 or older. All attributes are numeric, and their units vary from attribute to attribute.

Each record has a class value that indicates whether the patient suffered an onset of diabetes within 5 years of when the measurements were taken (1) or not (0).

This is a standard dataset that has been studied a lot in machine learning literature. A good prediction accuracy is 70%-76%.

## Naive Bayes Algorithm Tutorial

This tutorial is broken down into the following steps:

1. **Handle Data**: Load the data from CSV file and split it into training and test datasets.
2. **Summarize Data**: summarize the properties in the training dataset so that we can calculate probabilities and make predictions.
3. **Make a Prediction**: Use the summaries of the dataset to generate a single prediction.
4. **Make Predictions**: Generate predictions given a test dataset and a summarized training dataset.
5. **Evaluate Accuracy**: Evaluate the accuracy of predictions made for a test dataset as the percentage correct out of all predictions made.
6. **Tie it Together**: Use all of the code elements to present a complete and standalone implementation of the Naive Bayes algorithm.

## 1. Handle data
The first thing we need to do is load our data file. The data is in CSV format without a header line or any quotes. We can open the file with the open function and read the data lines using the reader function in the csv module.

We also need to convert the attributes that were loaded as strings into numbers that we can work with them. Below is the **loadCsv()** function for loading the Pima indians dataset.

In [1]:
import scala.collection.mutable.ArrayBuffer

def loadCsv(filename: String): List[List[Double]] = {
    val bufferedSource = io.Source.fromFile(filename)
    val rows = ArrayBuffer[List[Double]]()
    for (line <- bufferedSource.getLines) {
        rows += line.split(",").map(_.trim).map(_.toDouble).toList
    }
    bufferedSource.close
    rows.toList
}

import scala.collection.mutable.ArrayBuffer


defined function loadCsv

We can test this function by loading the pima indians dataset and printing the number of data instances that were loaded.

In [3]:
val filename = "pima-indians-diabetes.data.csv"
print(s"Loaded data file $filename with ${loadCsv(filename).length} rows")

Loaded data file pima-indians-diabetes.data.csv with 768 rows

filename: String = "pima-indians-diabetes.data.csv"

Next we need to split the data into a training dataset that Naive Bayes can use to make predictions and a test dataset that we can use to evaluate the accuracy of the model. We need to split the data set randomly into train and datasets with a ratio of 67% train and 33% test (this is a common ratio for testing an algorithm on a dataset).

Below is the **splitDataset()** function that will split a given dataset into a given split ratio.

In [3]:
import util.Random

def splitDataset(dataset: List[List[Double]], splitRatio: Double): (List[List[Double]], List[List[Double]]) = {
    val shuffled = Random.shuffle(dataset)
    shuffled.splitAt((dataset.length * splitRatio).toInt)
}

import util.Random


defined function splitDataset

## 2. Summarize Data
The naive bayes model is comprised of a summary of the data in the training dataset. This summary is then used when making predictions.

The summary of the training data collected involves the mean and the standard deviation for each attribute, by class value. For example, if there are two class values and 7 numerical attributes, then we need a mean and standard deviation for each attribute (7) and class value (2) combination, that is 14 attribute summaries.

These are required when making predictions to calculate the probability of specific attribute values belonging to each class value.

We can break the preparation of this summary data down into the following sub-tasks:

1. Separate Data By Class
2. Calculate Mean
3. Calculate Standard Deviation
4. Summarize Dataset
5. Summarize Attributes By Class

### Separate Data By Class
The first task is to separate the training dataset instances by class value so that we can calculate statistics for each class. We can do that by creating a map of each class value to a list of instances that belong to that class and sort the entire dataset of instances into the appropriate lists.

The **separateByClass()** function below does just this.

In [1]:
def separateByClass(dataset: List[List[Double]]): Map[Double, List[List[Double]]] = dataset.groupBy(_.last)

defined function separateByClass

You can see that the function assumes that the last attribute (-1) is the class value. The function returns a map of class values to lists of data instances.

We can test this function with some sample data, as follows:

In [2]:
val dataset = List(List(1.0, 20.0, 1.0), List(2.0, 21.0, 0.0), List(3.0, 22.0, 1.0))
val separated = separateByClass(dataset)
print(s"Separated instances: $separated")

Separated instances: Map(1.0 -> List(List(1.0, 20.0, 1.0), List(3.0, 22.0, 1.0)), 0.0 -> List(List(2.0, 21.0, 0.0)))

dataset: List[List[Double]] = List(List(1.0, 20.0, 1.0), List(2.0, 21.0, 0.0), List(3.0, 22.0, 1.0))
separated: Map[Double, List[List[Double]]] = Map(
  1.0 -> List(List(1.0, 20.0, 1.0), List(3.0, 22.0, 1.0)),
  0.0 -> List(List(2.0, 21.0, 0.0))
)

### Calculate Mean
We need to calculate the mean of each attribute for a class value. The mean is the central middle or central tendency of the data, and we will use it as the middle of our gaussian distribution when calculating probabilities.

We also need to calculate the standard deviation of each attribute for a class value. The standard deviation describes the variation of spread of the data, and we will use it to characterize the expected spread of each attribute in our Gaussian distribution when calculating probabilities.

The standard deviation is calculated as the square root of the variance. The variance is calculated as the average of the squared differences for each attribute value from the mean. Note we are using the N-1 method, which subtracts 1 from the number of attribute values when calculating the variance.

In [3]:
// TODO: validate this implementation, is the python one better?
def calcMeanStd(x: List[Double]): (Double, Double) = {
    @scala.annotation.tailrec
    def meanStd(x: List[Double], mu: Double, Q: Double, count: Int): (Double, Double) = {
        if (count >= x.length) (mu, Math.sqrt(Q / x.length))
        else {
            val newCount = count + 1
            val newMu = x(count) / newCount + mu * (1.0 - 1.0 / newCount)
            val newQ = Q + (x(count) - mu) * (x(count) - newMu)
            meanStd(x, newMu, newQ, newCount)   
        }        
    }

    meanStd(x, 0.0, 0.0, 0)    
}

def mean(numbers: List[Double]): Double = calcMeanStd(numbers)._1
def stdev(numbers: List[Double]): Double = calcMeanStd(numbers)._2

defined function calcMeanStd
defined function mean
defined function stdev

We can test this by taking the mean of the numbers from 1 to 5.

In [4]:
val numbers = List(1.0, 2.0, 3.0, 4.0, 5.0)
val (mean, stdev) = calcMeanStd(numbers)
print(s"Summary of $numbers: mean=$mean, stdev=$stdev")

Summary of List(1.0, 2.0, 3.0, 4.0, 5.0): mean=3.0, stdev=1.4142135623730951

numbers: List[Double] = List(1.0, 2.0, 3.0, 4.0, 5.0)
mean: Double = 3.0
stdev: Double = 1.4142135623730951

In [6]:
def summarize(dataset: List[List[Double]]): List[(Double, Double)] = dataset.transpose.dropRight(1).map(calcMeanStd)

defined function summarize

In [7]:
def summarizeByClass(dataset: List[List[Double]]): Map[Double, List[(Double, Double)]] = separateByClass(dataset).map {
    case (a, b) => (a, summarize(b))
}

defined function summarizeByClass

In [8]:
import scala.math.{exp, pow, sqrt, Pi}
def calculateProbability(x: Double, mean: Double, stdev: Double): Double = {
    val exponent = exp(-(pow(x - mean, 2) / (2 * pow(stdev, 2))))
    (1 / (math.sqrt(2 * Pi) * stdev)) * exponent
}

import scala.math.{exp, pow, sqrt, Pi}

defined function calculateProbability

In [9]:
def calculateClassProbabilities(summaries: Map[Double, List[(Double, Double)]], inputVector: List[Double]) = {
    summaries.map{ case (classValue, classSummaries) => (classValue, classSummaries.foldLeft(1.0) {
        (acc, tup) => calculateProbability(inputVector.head, tup._1, tup._2)
    } ) }
}

defined function calculateClassProbabilities

In [10]:
def predict(summaries: Map[Double, List[(Double, Double)]], inputVector: List[Double]) = {
    calculateClassProbabilities(summaries, inputVector).maxBy(_._2)._1
}

defined function predict

In [11]:
def getPredictions(summaries: Map[Double, List[(Double, Double)]], inputVector: List[List[Double]]) = {
    for (iv <- inputVector) yield predict(summaries, iv)
}

defined function getPredictions

In [12]:
def getAccuracy(testSet: List[List[Double]], predictions: List[Double]): Double = {
    val correct = for ( (data: List[Double], prediction: Double) <- testSet zip predictions) yield data.last == prediction
    (correct.filter(_ == true).length / testSet.length.toDouble) * 100
}

defined function getAccuracy

In [13]:
val testSet = List(List(1.0, 1.0, 1.0, 0.0), List(2.0, 2.0, 2.0, 0.0), List(3.0, 3.0, 3.0, 1.0))
val predictions = List(0.0, 0.0, 0.0)
getAccuracy(testSet, predictions)

testSet: List[List[Double]] = List(
  List(1.0, 1.0, 1.0, 0.0),
  List(2.0, 2.0, 2.0, 0.0),
  List(3.0, 3.0, 3.0, 1.0)
)
predictions: List[Double] = List(0.0, 0.0, 0.0)
res12_2: Double = 66.66666666666666

In [24]:
def main() = {
    val filename = "pima-indians-diabetes.data.csv"
    val splitRatio = 0.67
    val dataset = loadCsv(filename)
    val (trainingSet, testSet) = splitDataset(dataset, splitRatio)
    print(s"Split ${dataset.length} rows into train=${trainingSet.length} and test=${testSet.length} rows\n")
    val summaries = summarizeByClass(trainingSet)
    val predictions = getPredictions(summaries, testSet)
    val accuracy = getAccuracy(testSet, predictions)
    print(s"Accuracy: $accuracy")
}

defined function main

In [25]:
main()

Split 768 rows into train=514 and test=254 rows
Accuracy: 68.11023622047244